# Agent A: Personal Assistant

Our first real agent! It has two tools:
- **Calculator** — evaluates math expressions (a real tool, not hardcoded!)
- **Clock** — returns the current date and time

This uses everything from exercises 1-6: API calls, system prompts, tools, and the agentic loop.

## Step 1: Setup

In [ ]:
import json
import openai
from datetime import datetime

client = openai.OpenAI()

## Step 2: Define the system prompt

Notice we tell the model to **always** use the calculator for math. Without this, it might try to do arithmetic itself (and get it wrong).

In [ ]:
SYSTEM_PROMPT = """You are a helpful personal assistant. You have access to a calculator and a clock.
Use the calculator for any math — never do arithmetic in your head.
Be concise and friendly."""

## Step 3: Define tools and the dispatcher

The calculator uses Python's `eval()` on a restricted character set. The clock uses `datetime.now()`.

These are **real** tools — they compute actual results, not hardcoded strings.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Evaluate a mathematical expression. Supports +, -, *, /, **, parentheses, and common math.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The math expression to evaluate, e.g. '(12 + 34) * 5'",
                    }
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_time",
            "description": "Get the current date, day of week, and time",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
]


def call_tool(name, input):
    if name == "calculator":
        try:
            allowed = set("0123456789+-*/.() ")
            expr = input["expression"]
            if all(c in allowed for c in expr):
                return str(eval(expr))
            else:
                return "Error: expression contains invalid characters"
        except Exception as e:
            return f"Error: {e}"
    elif name == "get_time":
        return datetime.now().strftime("%A, %Y-%m-%d %H:%M:%S")
    return "Unknown tool"

## Step 4: Run the agent

Same agentic loop from exercise 6, now with a system prompt and real tools.

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What is 2847 divided by 13? Also what day of the week is it?"},
]

while True:
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools,
    )

    message = response.choices[0].message
    messages.append(message)

    if message.tool_calls:
        for tool_call in message.tool_calls:
            args = json.loads(tool_call.function.arguments)
            result = call_tool(tool_call.function.name, args)
            print(f"  [{tool_call.function.name}] {args} → {result}")
            messages.append(
                {"role": "tool", "tool_call_id": tool_call.id, "content": result}
            )
    else:
        print(f"\nAssistant: {message.content}")
        break

## Try it yourself

Change the user message above. Some ideas:
- "What is 15% tip on a 84.50 dollar bill?"
- "What time is it right now?"
- "If I earn 45 dollars per hour and work 37.5 hours a week, what's my monthly income?"

For an interactive chat version, run:
```bash
uv run python foundation/openai/07_agent_personal/agent_personal.py
```